In [1]:
# imports
import pandas as pd
import json
import requests
import folium
print ("libraries imported")

libraries imported


#### Loading stations dataframe

In [2]:
stations_df = pd.read_csv('stations.csv')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

### I CREATED A FUNCTION FOR FOURSQUARE TO ITERATE OVER THE DATAFRAME OF STATIONS

In [3]:
def getNearbyPOI(latitude, longitude):
    
    url = "https://api.foursquare.com/v3/places/search"

    params = {
    "radius":"1000",
  	"ll": f"{latitude},{longitude}",
  	"open_now": "true",
  	"sort":"DISTANCE"
    }

    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3KKRisSK1dwYHI3BlvESFIU4LjVkyURq0ipp9jNCHwxc="
    }

    response = requests.request("GET", url, params=params, headers=headers)

    r_json=json.loads(response.text)
    
    return r_json



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

### AFTER WORKING WITH A MENTOR, I MANAGED TO CREATE A DATAFRAME WITH BOTH THE STATION DETAILS (NAME, LAT, LONG, USAGE) AND THE POINT OF INTEREST DETAILS (NAME, DISTANCE, RATING, ADDRESS) ALL WRITTEN TO A DATAFRAME "df_fspoi" AS DETAILED BELOW¶

In [4]:
df_fspoi = pd.DataFrame(columns=['station','usage','total_bikes','poi_name', 'poi_distance', 'poi_address'])

for index, station in stations_df.iterrows():
    myjson=getNearbyPOI(station['latitude'],station['longitude'])
    for business in myjson['results']:  
        fs_info_data = {
                "name": business['name'],
                "distance": business['distance'],
                "address": business['location']['formatted_address']}
        row = [station['name'],station['usage_percentage'],station['total_bikes'],fs_info_data['name'],fs_info_data['distance'],fs_info_data['address']]
        df_fspoi.loc[len(df_fspoi)] = row

Put your parsed results into a DataFrame

In [5]:
df_fspoi

,station,usage,total_bikes,poi_name,poi_distance,poi_address
0,Hollywood North Beach,0.636364,11,Hollywood North Beach Park,9,"3601 N Ocean Dr, Hollywood, FL 33019"
1,Hollywood North Beach,0.636364,11,Food King,414,"3111 N Surf Rd, Hollywood, FL 33019"
2,Hollywood North Beach,0.636364,11,Oceans 13 Sports Bar & Grill,432,"3111 N Surf Rd, Hollywood, FL 33019"
3,Hollywood North Beach,0.636364,11,Anne Kolb Nature Center,614,"751 Sheridan St, Hollywood, FL 33019"
4,Hollywood North Beach,0.636364,11,Marine Environmental Education Center at the C...,696,"4414 N Surf Rd, Hollywood, FL 33019"
...,...,...,...,...,...,...
188,Willingham Park,0.769231,13,Art of Petals,203,"2200 NE 33rd Ave, Fort Lauderdale, FL 33305"
189,Willingham Park,0.769231,13,Lu Deaner Park,214,"2125 NE 33rd Ave, Fort Lauderdale, FL 33305"
190,Willingham Park,0.769231,13,Calypso Villas by the Beach,444,"2419 NE 33rd Ave (33rd ave.), Fort Lauderdale,..."
191,Willingham Park,0.769231,13,Sandra Lee Apartments Motel,540,"2307 NE 33rd Ave, Fort Lauderdale, FL 33305"


In [6]:
# I'm saving the dataframe to CSV for the next section
df_fspoi.to_csv('FourSquarePOI.csv', index=False)

### Descriptive Statistics

In [7]:
df_fspoi.shape

(193, 6)

In [8]:
df_fspoi.info

<bound method DataFrame.info of                    station     usage  total_bikes  \
0    Hollywood North Beach  0.636364           11   
1    Hollywood North Beach  0.636364           11   
2    Hollywood North Beach  0.636364           11   
3    Hollywood North Beach  0.636364           11   
4    Hollywood North Beach  0.636364           11   
..                     ...       ...          ...   
188        Willingham Park  0.769231           13   
189        Willingham Park  0.769231           13   
190        Willingham Park  0.769231           13   
191        Willingham Park  0.769231           13   
192        Willingham Park  0.769231           13   

                                              poi_name  poi_distance  \
0                           Hollywood North Beach Park             9   
1                                            Food King           414   
2                         Oceans 13 Sports Bar & Grill           432   
3                              Anne Kolb Na

### Data cleaning - checking for duplicates in points of interest is to be expected, since some stations are within 2000 metres of others.  Stations have no duplicates as checked earlier

### Data cleaning - Checking for Null Values¶

In [9]:
print(df_fspoi.isnull().sum())

station         0
usage           0
total_bikes     0
poi_name        0
poi_distance    0
poi_address     0
dtype: int64


In [ ]:
#no null values
# we won't check for outliers as thats not relevant for foursquare points of interest

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [55]:
# imports
import pandas as pd
import json
import requests
print ("libraries imported")

libraries imported


In [56]:
def getNearbyYelp(latitude, longitude):
    
    
    # Define my API Key, My Endpoint, and My Header
    API_KEY = '4RFCgRndWzUphm4vgIsFet58SboTR76bj7eTDXFL5wwhS3_E6RZ8qnye_qVdiaXqgr1FbR1Y9QxzruU6a6AlcbjRGAtsgaDfis7WsRQY9GTRf4c2AVPibpzdt2P4ZHYx'
    ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
    HEADERS = {'Authorization': 'bearer %s' % API_KEY}
   
    
    # BUSINESS SEARCH PARAMETERS 
    PARAMETERS = {'latitude': f"{latitude}",
              'longitude': f"{longitude}",
              'radius': 1000}   
    
    
    # Make a request to the Yelp API
    response = requests.get(url = ENDPOINT,
                        params = PARAMETERS,
                        headers = HEADERS)   
    
    yelp_data = response.json() 
    
    return yelp_data



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [57]:
df_yelp_poi = pd.DataFrame(columns=['station','usage','total_bikes','poi_name', 'poi_distance', 'poi_address'])

for index, station in stations_df.iterrows():
    yelpjson=getNearbyYelp(station['latitude'],station['longitude'])
    for business in yelpjson['businesses']:
        yelp_info_data = {
                "name": business['name'],
                "distance": business['distance'],
                "address": business['location']['address1']
                }
        row = [station['name'],station['usage_percentage'],station['total_bikes'],yelp_info_data['name'],yelp_info_data['distance'],yelp_info_data['address']]
        df_yelp_poi.loc[len(df_yelp_poi)] = row

In [58]:
df_yelp_poi.shape

(377, 6)

In [59]:
df_yelp_poi.info

<bound method DataFrame.info of                    station     usage  total_bikes  \
0    Hollywood North Beach  0.636364           11   
1    Hollywood North Beach  0.636364           11   
2    Hollywood North Beach  0.636364           11   
3    Hollywood North Beach  0.636364           11   
4    Hollywood North Beach  0.636364           11   
..                     ...       ...          ...   
372        Willingham Park  0.857143           14   
373        Willingham Park  0.857143           14   
374        Willingham Park  0.857143           14   
375        Willingham Park  0.857143           14   
376        Willingham Park  0.857143           14   

                           poi_name  poi_distance               poi_address  
0        Hollywood North Beach Park     38.693472           3601 N Ocean Dr  
1    Latitudes Restaurant Hollywood    794.731229       2501 North Ocean Dr  
2      Oceans 13 Sports Bar & Grill    451.282571            3111 N Surf Rd  
3           Anne Ko

### Data cleaning - checking for duplicates in points of interest is to be expected, since some stations are within 2000 metres of others. Stations have no duplicates as checked earlier

### Data cleaning - checking for null values

In [60]:
print(df_yelp_poi.isnull().sum())

station         0
usage           0
total_bikes     0
poi_name        0
poi_distance    0
poi_address     0
dtype: int64


In [119]:
#no null values
# we won't check for outliers as thats not relevant for foursquare points of interest

Put your parsed results into a DataFrame

In [61]:
df_yelp_poi

,station,usage,total_bikes,poi_name,poi_distance,poi_address
0,Hollywood North Beach,0.636364,11,Hollywood North Beach Park,38.693472,3601 N Ocean Dr
1,Hollywood North Beach,0.636364,11,Latitudes Restaurant Hollywood,794.731229,2501 North Ocean Dr
2,Hollywood North Beach,0.636364,11,Oceans 13 Sports Bar & Grill,451.282571,3111 N Surf Rd
3,Hollywood North Beach,0.636364,11,Anne Kolb Nature Center,685.543129,751 Sheridan St
4,Hollywood North Beach,0.636364,11,Keating Beach,62.695769,1175-1199 Surf Rd
...,...,...,...,...,...,...
372,Willingham Park,0.857143,14,Willingham Park,78.929008,2100 N Atlantic Blvd
373,Willingham Park,0.857143,14,Lu Deaner Park,217.537675,2125 NE 33rd Ave
374,Willingham Park,0.857143,14,Loggerhead Park,741.500463,3698 NE 27th St
375,Willingham Park,0.857143,14,Moreth House,508.069849,3000 NE 19th St


In [62]:
# I'm saving the dataframe to CSV for the next section
df_yelp_poi.to_csv('YelpPOI.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

### Generally yelp seems to provide a higher quality of information considering they give a rating to the businesses involved, also they simply have many more businesses tracked as points of interest in Fort Lauderdale than foursquare does. 

Get the top 10 restaurants according to their rating

In [155]:
    import pandas as pd
    import json
    
    # Define my API Key, My Endpoint, and My Header
    API_KEY = '4RFCgRndWzUphm4vgIsFet58SboTR76bj7eTDXFL5wwhS3_E6RZ8qnye_qVdiaXqgr1FbR1Y9QxzruU6a6AlcbjRGAtsgaDfis7WsRQY9GTRf4c2AVPibpzdt2P4ZHYx'
    ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
    HEADERS = {'Authorization': 'bearer %s' % API_KEY}
   
    
    # BUSINESS SEARCH PARAMETERS 
    PARAMETERS = {'location': "Fort Lauderdale",
                  'term': "restaurants",
                  'sort_by': 'rating',
                  'limit': 10}
    
    
    # Make a request to the Yelp API
    response = requests.get(url = ENDPOINT,
                        params = PARAMETERS,
                        headers = HEADERS)   
    
    top10 = response.json()
    
    
    for business in top10['businesses']:
                top10_data = {
                "name": business['name'],
                "address": business['location']['address1']
                }
                print (top10_data)
                
 

{'name': 'Poblano South', 'address': '1315 SW First Ct'}
{'name': 'Grapes Wine Cafe & Market', 'address': '3311 N Ocean Blvd'}
{'name': 'Newbury Salads', 'address': '75 Main St'}
{'name': 'Mr Bill BBQ', 'address': ''}
{'name': 'Island Flavor cafe', 'address': '7958 W McNab Rd N'}
{'name': 'Yumbo Poke', 'address': '3325 E Oakland Park Blvd'}
{'name': 'Pots & Pans Cafe', 'address': '952 NE 62nd St'}
{'name': 'La Villa', 'address': '717 Breakers Ave'}
{'name': 'Danlu Cafe', 'address': '200 E Broward Blvd'}
{'name': 'Luka Restaurant', 'address': '244 S University Dr'}


### Top 10 restaurants according to Yelp ratings for Fort Lauderdale are listed above.  Yelp documentation indicates that the: "rating sort is NOT strictly sorted by the rating value, but by an adjusted rating value that takes into account the number of ratings, similar to a Bayesian average. This is to prevent skewing results to businesses with a single review".  Due to this (favourable) detail of the Yelp rating sort, I felt is was suitable to just show the top 10 results.